# Generative Adversarial Networks (GAN)

## Objectives
- To learn about GAN
- To train a model using GAN
- To train a model using cGAN
- To make comparision

## Theory
**GAN:** using deep learning to train generative models. It has the archetcture of:
- **Generator:** responsible for generating new plausabile examples that ideally indistingishable from real eaxamples in the dataset.
- **Discriminator:** is responsible for classifting a given image as either real or fake.

**Adversial:** zero-sum : improvemnets in the discrimiter come at the cost of reducing the capability of the generator.

Image synthesis (GAN is effective).

In the case that we make use of the labels:
- Improve the GAN.
- Targeted Generation.

**The limitation of GAN:** (There is a relationship between points in the latent space to the generated images, but this relationship is complex and hard to map) it might generate random image from the domain.

If we trainied it in such way that the descriminator and the generators are conditioned on the class label as an alternative. An example is genertaed number 9 from the digits dataset. It's called **cGAN**

**Deep convolutional GAN (DCGAN):** stable traiing of GAN models. 
The conditional traininng of the DCGAN-based models may be reffered to as cDCGAN). 

Best practice to incorpiirate the class labels into the discriminator is using the embeding layer -> FC -> Linear activation.

Neccessary imports

In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))